<a href="https://colab.research.google.com/github/X7OH/TicTacToe/blob/main/ProyectoML1erParcialF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from google.colab import userdata
import torch
from transformers import Trainer, TrainingArguments

# Obtener el token desde los secretos
hf_token = userdata.get('HF_TOKEN')

# Nombre del modelo
model_name = "bigscience/bloom-560m"

# Verificar si la GPU está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando el dispositivo: {device}")

# Cargar el tokenizador y el modelo usando el token desde secrets
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)
model = AutoModelForCausalLM.from_pretrained(model_name, token=hf_token)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from google.colab import userdata

# Obtener tu token de Hugging Face desde Google Colab
hf_token2 = userdata.get('EHF_TOKEN2')

# Nombre del modelo
model_name = "distilbert/distilgpt2"  # Puedes cambiarlo por otro modelo compatible

# Cargar el tokenizador y modelo con el token de Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=hf_token)
model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=hf_token)

In [ ]:
import random
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Clase TicTacToe para manejar el juego
class TicTacToe:
    def __init__(self):
        self.board = [' ' for _ in range(9)]  # 9 celdas vacías
        self.current_turn = 'X'  # 'X' empieza primero

    def _printBoard(self):
        for i in range(0, 9, 3):
            print(f"{self.board[i]} | {self.board[i+1]} | {self.board[i+2]}")
            if i < 6:
                print("---------")

    def _checkWin(self, mark):
        win_conditions = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # Filas
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # Columnas
            [0, 4, 8], [2, 4, 6]             # Diagonales
        ]
        for condition in win_conditions:
            if all(self.board[i] == mark for i in condition):
                return True
        return False

    def _getEmptyCells(self):
        return [i for i, cell in enumerate(self.board) if cell == ' ']

    def playMove(self, position, mark):
        if self.board[position] == ' ':
            self.board[position] = mark
            return True
        return False

    def isFull(self):
        return ' ' not in self.board

def aiMove(model, tokenizer, device, board, mark):
    # Convertir el estado del tablero en un formato de texto para la IA
    board_state = ''.join(board)
    prompt = f"El tablero de TicTacToe es: {board_state}. ¿Dónde debe colocar la ficha '{mark}'?"

    # Tokenizar la entrada
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generar la respuesta del modelo
    with torch.no_grad():
        outputs = model.generate(inputs['input_ids'], max_length=50)

    # Decodificar la salida
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("Respuesta de la IA:", response)

    # Interpretar la respuesta para obtener la jugada
    move = None
    for i in range(9):
        if f"{i}" in response:
            move = i
            break

    # Validar si la posición está dentro del rango permitido
    if move is not None and 0 <= move < 9 and board[move] == ' ':
        return move
    else:
        print("Movimiento inválido, eligiendo una posición aleatoria.")
        return random.choice([i for i in range(9) if board[i] == ' '])


# Función evaluadora
def evaluateMove(board, position, mark):
    if board[position] == ' ':
        return "Válida"
    return "Inválida"

from google.colab import userdata

# Inicialización de modelos
hf_token = userdata.get('HF_TOKEN')  # jugador 1
hf_token2 = userdata.get('HF_TOKEN2')  # jugador 2
ehf_token = userdata.get('EHF_TOKEN') # evaluador

model_name_player1 = "bigscience/bloom-560m"
model_name_player2 = "TinyLlama/TinyLlama_v1.1"
model_name_evaluator = "distilgpt2"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Cargar modelos
tokenizer_p1 = AutoTokenizer.from_pretrained(model_name_player1, token=hf_token)
model_p1 = AutoModelForCausalLM.from_pretrained(model_name_player1, token=hf_token).to(device)

tokenizer_p2 = AutoTokenizer.from_pretrained(model_name_player2, token=hf_token2)
model_p2 = AutoModelForCausalLM.from_pretrained(model_name_player2, token=hf_token2).to(device)

tokenizer_eval = AutoTokenizer.from_pretrained(model_name_evaluator, token=ehf_token)
model_eval = AutoModelForCausalLM.from_pretrained(model_name_evaluator, token=ehf_token).to(device)


# def evaluateMove(model, tokenizer, device, board, position, mark):
#     # Convertir el estado del tablero en un formato de texto para la IA
#     board_state = ''.join(board)
#     prompt = f"El tablero de TicTacToe es: {board_state}. ¿Cuál es la próxima jugada válida para '{mark}'?"

#     # Tokenizar la entrada
#     inputs = tokenizer(prompt, return_tensors="pt").to(device)

#     # Generar la respuesta del modelo
#     with torch.no_grad():
#         outputs = model.generate(inputs['input_ids'], max_length=50)

#     # Decodificar la salida
#     response = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     print("Respuesta de la IA Evaluadora:", response)

#     # Interpretar la respuesta para determinar si la jugada es válida
#     if "válida" in response.lower():
#         return "Válida"
#     else:
#         return "Inválida"
def evaluateMove(model, tokenizer, device, board, position, mark):
    # Convertir el estado del tablero en un formato de texto para la IA
    board_state = ''.join(board)

    # Crear un prompt que pregunte si la jugada en la posición actual es válida
    prompt = f"El tablero de TicTacToe es: {board_state}. ¿Cuál es la próxima jugada válida para '{mark}'?"

    # Tokenizar la entrada
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generar la respuesta del modelo
    with torch.no_grad():
        outputs = model.generate(inputs['input_ids'], max_length=50)

    # Decodificar la salida
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("Respuesta de la IA Evaluadora:", response)

    # Interpretar la respuesta para determinar si la jugada es válida
    if "válida" in response.lower():
        return "Válida"
    else:
        return "Inválida"

# Simular partidas
game = TicTacToe()
n_games = 10
wins_p1, wins_p2 = 0, 0

for game_num in range(1, n_games + 1):
    print(f"\nPartida {game_num}:")
    game.board = [' ' for _ in range(9)]
    game.current_turn = 'X'

    while not game.isFull():
        if game.current_turn == 'X':
            print("Turno del jugador IA 1 (X):")
            move = aiMove(model_p1, tokenizer_p1, device, game.board, 'X')
            evaluation = evaluateMove(model_eval, tokenizer_eval, device, game.board, move, 'X')
            # print(f"Evaluación de la jugada: {evaluation}")
            if evaluation == "Válida" and game.playMove(move, 'X'):
                game._printBoard()
                if game._checkWin('X'):
                    print("¡Jugador 1 (X) ha ganado!")
                    wins_p1 += 1
                    break
            game.current_turn = 'O'

        elif game.current_turn == 'O':
            print("Turno del jugador IA 2 (O):")
            move = aiMove(model_p2, tokenizer_p2, device, game.board, 'O')
            evaluation = evaluateMove(model_eval, tokenizer_eval, device, game.board, move, 'O')
            # print(f"Evaluación de la jugada: {evaluation}")
            if evaluation == "Válida" and game.playMove(move, 'O'):
                game._printBoard()
                if game._checkWin('O'):
                    print("¡Jugador 2 (O) ha ganado!")
                    wins_p2 += 1
                    break
            game.current_turn = 'X'

# Mostrar estadísticas finales
print("\nEstadísticas:")
print(f"Jugador 1 (X) - Victorias: {wins_p1}")
print(f"Jugador 2 (O) - Victorias: {wins_p2}")
print(f"Porcentaje de victorias del Jugador 1: {wins_p1 / n_games * 100}%")
print(f"Porcentaje de victorias del Jugador 2: {wins_p2 / n_games * 100}%")

In [ ]:
import random

# Clase TicTacToe para manejar el juego
class TicTacToe:
    def __init__(self):
        self.board = [' ' for _ in range(9)]  # 9 celdas vacías
        self.current_turn = 'X'  # 'X' empieza primero

    def _printBoard(self):
        for i in range(0, 9, 3):
            print(f"{self.board[i]} | {self.board[i+1]} | {self.board[i+2]}")
            if i < 6:
                print("---------")

    def _checkWin(self, mark):
        win_conditions = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # Filas
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # Columnas
            [0, 4, 8], [2, 4, 6]             # Diagonales
        ]
        for condition in win_conditions:
            if all(self.board[i] == mark for i in condition):
                return True
        return False

    def _getEmptyCells(self):
        return [i for i, cell in enumerate(self.board) if cell == ' ']

    def playMove(self, position, mark):
        if self.board[position] == ' ':
            self.board[position] = mark
            return True
        return False

    def isFull(self):
        return ' ' not in self.board


# Función principal para simular juegos
def simulate_games(n_games, player1_model, player2_model, evaluator_model):
    wins_p1 = 0
    wins_p2 = 0
    invalid_moves = 0
    total_moves = 0

    for game in range(n_games):
        game_instance = TicTacToe()
        game_over = False

        print(f"\n--- Juego {game + 1} ---")
        while not game_over:
            # Alternar entre jugadores
            current_player_model = player1_model if game_instance.current_turn == 'X' else player2_model
            opponent = 'O' if game_instance.current_turn == 'X' else 'X'

            # Obtener movimiento del jugador
            empty_cells = game_instance._getEmptyCells()
            if not empty_cells:
                break  # Tablero lleno

            move = int(current_player_model.play_turn(game_instance.board, game_instance.current_turn, empty_cells))

            # Validar jugada con el modelo evaluador
            is_valid = evaluator_model.evaluate_move(game_instance.board, move, game_instance.current_turn)
            total_moves += 1

            if not is_valid:
                print(f"Jugador {game_instance.current_turn} realizó una jugada inválida en {move}.")
                invalid_moves += 1
                # Asignar victoria al oponente
                if game_instance.current_turn == 'X':
                    wins_p2 += 1
                else:
                    wins_p1 += 1
                game_over = True
                break

            # Si la jugada es válida, actualiza el tablero
            game_instance.playMove(move, game_instance.current_turn)

            # Revisar si hay un ganador
            if game_instance._checkWin(game_instance.current_turn):
                print(f"¡Jugador {game_instance.current_turn} ganó!")
                if game_instance.current_turn == 'X':
                    wins_p1 += 1
                else:
                    wins_p2 += 1
                game_over = True
            elif game_instance.isFull():
                print("El juego terminó en empate.")
                game_over = True
            else:
                # Cambiar turno
                game_instance.current_turn = opponent


def evaluator_prompt(board, move, player):
    """
    Genera el prompt para el modelo evaluador basado en el estado del tablero y la jugada.
    """
    board_str = "\n".join([" | ".join(board[i:i+3]) for i in range(0, 9, 3)])
    prompt = (
        f"Estado del tablero:\n{board_str}\n\n"
        f"El jugador '{player}' propone jugar en la posición {move}.\n"
        f"Una jugada válida debe ser un número entre 0 y 8 y la celda correspondiente debe estar vacía.\n"
        f"¿Es esta una jugada válida considerando el tablero actual? "
        f"Responde con 'válida' o 'inválida'."
    )
    return prompt

def create_player_prompt(board, current_player, opponent_player):

    # Convertir el tablero en una representación legible
    board_str = '\n'.join([
        f"{board[i]} | {board[i+1]} | {board[i+2]}" if i < 6 else f"{board[i]} | {board[i+1]} | {board[i+2]}"
        for i in range(0, 9, 3)
    ])

    # Generar el prompt para el jugador
    prompt = f"""
    Eres un jugador experto de Tic Tac Toe. El tablero actual es el siguiente:
    {board_str}

    Tu jugada es como el jugador {current_player}. El jugador {opponent_player} ya ha jugado.

    Usa este estado del tablero para tomar una decisión sobre tu siguiente movimiento.
    Tu tarea es escoger una posición vacía entre 0 y 8 para hacer tu jugada.
    """

    return prompt

import torch
from transformers import pipeline
from google.colab import userdata


# # Inicialización de los modelos
# hf_token = userdata.get('HF_TOKEN')  # jugador 1
# hf_token2 = userdata.get('HF_TOKEN2')  # jugador 2
# ehf_token = userdata.get('EHF_TOKEN') # evaluador

# # Usando Hugging Face pipeline para cargar los modelos
# model_name_player1 = "bigscience/bloom-560m"
# model_name_player2 = "TinyLlama/TinyLlama_v1.1"
# model_name_evaluator = "distilgpt2"

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Modelos de Jugadores (para hacer jugadas)
# player1_model = pipeline("text-generation", model=model_name_player1, tokenizer=model_name_player1, device=0 if torch.cuda.is_available() else -1)
# player2_model = pipeline("text-generation", model=model_name_player2, tokenizer=model_name_player2, device=0 if torch.cuda.is_available() else -1)

# # Modelo Evaluador (para validar jugadas)
# evaluator_model = pipeline("text-generation", model=model_name_evaluator, tokenizer=model_name_evaluator, device=0 if torch.cuda.is_available() else -1)

# def simulate_games(n_games, player1_model, player2_model, evaluator_model):
#     wins_p1 = 0
#     wins_p2 = 0
#     invalid_moves = 0
#     total_moves = 0

#     for game in range(n_games):
#         game_instance = TicTacToe()
#         game_over = False

#         print(f"\n--- Juego {game + 1} ---")
#         while not game_over:
#             # Alternar entre jugadores
#             current_player_model = player1_model if game_instance.current_turn == 'X' else player2_model
#             opponent = 'O' if game_instance.current_turn == 'X' else 'X'

#             # Obtener el prompt para el jugador actual
#             prompt = create_player_prompt(game_instance.board, game_instance.current_turn, opponent)

#             # Obtener el movimiento del jugador usando su modelo
#             move_response = current_player_model(prompt, max_new_tokens=10, num_return_sequences=1)


#             move = process_move(move_response[0]['generated_text'])  # Función que procesa la salida del modelo para extraer la jugada

#             # Validar la jugada con el modelo evaluador
#             eval_prompt = evaluator_prompt(game_instance.board, move, game_instance.current_turn)
#             eval_response = evaluator_model(eval_prompt, max_new_tokens=10, num_return_sequences=1)
#             is_valid = process_evaluation(eval_response[0]['generated_text'])  # Función que procesa la respuesta del evaluador

#             total_moves += 1

#             if not is_valid:
#                 print(f"Jugador {game_instance.current_turn} realizó una jugada inválida en {move}.")
#                 invalid_moves += 1
#                 # Terminar la partida sin victoria
#                 game_over = True
#                 break

#             # Si la jugada es válida, actualiza el tablero
#             game_instance.playMove(move, game_instance.current_turn)

#             # Revisar si hay un ganador
#             if game_instance._checkWin(game_instance.current_turn):
#                 print(f"¡Jugador {game_instance.current_turn} ganó!")
#                 if game_instance.current_turn == 'X':
#                     wins_p1 += 1
#                 else:
#                     wins_p2 += 1
#                 game_over = True
#             elif game_instance.isFull():
#                 print("El juego terminó en empate.")
#                 game_over = True
#             else:
#                 # Cambiar turno
#                 game_instance.current_turn = opponent

#     # Resumen de los resultados
#     print(f"\nResultados: \nJugador 1 (X) ganó {wins_p1} veces.")
#     print(f"Jugador 2 (O) ganó {wins_p2} veces.")
#     print(f"Juegos con jugadas inválidas: {invalid_moves}")
#     print(f"Juegos jugados en total: {total_moves}")


# # Función para procesar la salida del modelo y extraer la jugada
# def process_move(model_output):
#     # Aquí puedes ajustar la forma de extraer la jugada según cómo el modelo devuelva los datos
#     # Supongamos que el modelo devuelve un texto que contiene un número entre 0 y 8
#     move = None
#     for word in model_output.split():
#         if word.isdigit() and 0 <= int(word) <= 8:
#             move = int(word)
#             break
#     return move

# # Función para procesar la salida del evaluador y determinar si la jugada es válida
# def process_evaluation(eval_output):
#     # Supongamos que el modelo devuelve "válida" o "inválida"
#     if "válida" in eval_output.lower():
#         return True
#     elif "inválida" in eval_output.lower():
#         return False
#     return False


# # Ejecuta la simulación de 5 juegos
# simulate_games(5, player1_model, player2_model, evaluator_model)

# hf_token = userdata.get('HF_TOKEN')  # jugador 1
# hf_token2 = userdata.get('HF_TOKEN2')  # jugador 2
# ehf_token = userdata.get('EHF_TOKEN') # evaluador

# # Usando Hugging Face pipeline para cargar los modelos
# model_name_player1 = "bigscience/bloom-560m"
# model_name_player2 = "TinyLlama/TinyLlama_v1.1"
# model_name_evaluator = "distilgpt2"

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# player1_model = pipeline("text-generation", model=model_name_player1, tokenizer=model_name_player1, device=-1)  # Usar CPU
# player2_model = pipeline("text-generation", model=model_name_player2, tokenizer=model_name_player2, device=-1)  # Usar CPU
# evaluator_model = pipeline("text-generation", model=model_name_evaluator, tokenizer=model_name_evaluator, device=-1)  # Usar CPU

from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM


# Inicialización de modelos
hf_token = userdata.get('HF_TOKEN')  # jugador 1
hf_token2 = userdata.get('HF_TOKEN2')  # jugador 2
ehf_token = userdata.get('EHF_TOKEN') # evaluador

model_name_player1 = "bigscience/bloom-560m"
model_name_player2 = "TinyLlama/TinyLlama_v1.1"
model_name_evaluator = "distilgpt2"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Cargar modelos
tokenizer_p1 = AutoTokenizer.from_pretrained(model_name_player1, token=hf_token)
model_p1 = AutoModelForCausalLM.from_pretrained(model_name_player1, token=hf_token).to(device)

tokenizer_p2 = AutoTokenizer.from_pretrained(model_name_player2, token=hf_token2)
model_p2 = AutoModelForCausalLM.from_pretrained(model_name_player2, token=hf_token2).to(device)

tokenizer_eval = AutoTokenizer.from_pretrained(model_name_evaluator, token=ehf_token)
model_eval = AutoModelForCausalLM.from_pretrained(model_name_evaluator, token=ehf_token).to(device)

# # Función principal para simular juegos
# def simulate_games(n_games, player1_model, player2_model, evaluator_model):
#     wins_p1 = 0
#     wins_p2 = 0
#     invalid_moves = 0
#     total_moves = 0

#     for game in range(n_games):
#         game_instance = TicTacToe()
#         game_over = False

#         print(f"\n--- Juego {game + 1} ---")
#         while not game_over:
#             # current_player_model = player1_model if game_instance.current_turn == 'X' else player2_model
#             # opponent = 'O' if game_instance.current_turn == 'X' else 'X'

#             # # Obtener el prompt para el jugador actual
#             # prompt = create_player_prompt(game_instance.board, game_instance.current_turn, opponent)

#             # # Obtener el movimiento del jugador usando su modelo
#             # move_response = current_player_model(prompt, max_new_tokens=10, num_return_sequences=1)
#             # move = process_move(move_response[0]['generated_text'])
#             current_player_model = player1_model if game_instance.current_turn == 'X' else player2_model
#             current_tokenizer = tokenizer_p1 if game_instance.current_turn == 'X' else tokenizer_p2 # Select appropriate tokenizer
#             opponent = 'O' if game_instance.current_turn == 'X' else 'X'

#             # Obtener el prompt para el jugador actual
#             prompt = create_player_prompt(game_instance.board, game_instance.current_turn, opponent)

#             # Obtener el movimiento del jugador usando su modelo
#             # Tokenize the prompt
#             input_ids = current_tokenizer(prompt, return_tensors="pt").input_ids.to(device)

#             # Generate text
#             # Pass generation arguments to generate()
#             move_response = current_player_model.generate(input_ids, max_new_tokens=10, num_return_sequences=1)

#             # Decode the generated text
#             generated_text = current_tokenizer.decode(move_response[0], skip_special_tokens=True)
#             move = process_move(generated_text)


#             if move is None:
#                 print(f"Jugador {game_instance.current_turn} no pudo generar una jugada válida.")
#                 game_over = True
#                 break

#             # Validar la jugada con el modelo evaluador
#             eval_prompt = evaluator_prompt(game_instance.board, move, game_instance.current_turn)
#             eval_response = evaluator_model(eval_prompt, max_new_tokens=10, num_return_sequences=1)
#             is_valid = process_evaluation(eval_response[0]['generated_text'])

#             total_moves += 1

#             if not is_valid:
#                 print(f"Jugador {game_instance.current_turn} realizó una jugada inválida en {move}.")
#                 invalid_moves += 1
#                 game_over = True
#                 break

#             # Si la jugada es válida, actualiza el tablero
#             game_instance.playMove(move, game_instance.current_turn)

#             # Revisar si hay un ganador
#             if game_instance._checkWin(game_instance.current_turn):
#                 print(f"¡Jugador {game_instance.current_turn} ganó!")
#                 if game_instance.current_turn == 'X':
#                     wins_p1 += 1
#                 else:
#                     wins_p2 += 1
#                 game_over = True
#             elif game_instance.isFull():
#                 print("El juego terminó en empate.")
#                 game_over = True
#             else:
#                 # Cambiar turno
#                 game_instance.current_turn = opponent

#     # Resumen de los resultados
#     print(f"\nResultados: \nJugador 1 (X) ganó {wins_p1} veces.")
#     print(f"Jugador 2 (O) ganó {wins_p2} veces.")
#     print(f"Juegos con jugadas inválidas: {invalid_moves}")
#     print(f"Juegos jugados en total: {total_moves}")

# # Ejecuta la simulación de 5 juegos
# simulate_games(5, model_p1, model_p2, model_eval)
def simulate_games(n_games, player1_model, player2_model, evaluator_model):
    wins_p1 = 0
    wins_p2 = 0
    invalid_moves = 0
    total_moves = 0

    for game in range(n_games):
        game_instance = TicTacToe()
        game_over = False

        print(f"\n--- Juego {game + 1} ---")
        while not game_over:
            current_player_model = player1_model if game_instance.current_turn == 'X' else player2_model
            current_tokenizer = tokenizer_p1 if game_instance.current_turn == 'X' else tokenizer_p2
            opponent = 'O' if game_instance.current_turn == 'X' else 'X'

            # Obtener el prompt para el jugador actual
            prompt = create_player_prompt(game_instance.board, game_instance.current_turn, opponent)

            # Tokenizar el prompt
            input_ids = current_tokenizer(prompt, return_tensors="pt").input_ids.to(device)

            # Generar el movimiento
            move_response = current_player_model.generate(input_ids, max_new_tokens=10, num_return_sequences=1)

            # Decodificar el texto generado
            generated_text = current_tokenizer.decode(move_response[0], skip_special_tokens=True)
            move = process_move(generated_text)

            if move is None:
                print(f"Jugador {game_instance.current_turn} no pudo generar una jugada válida.")
                game_over = True
                break

            # Validar la jugada con el modelo evaluador
            eval_prompt = evaluator_prompt(game_instance.board, move, game_instance.current_turn)
            eval_input_ids = tokenizer_eval(eval_prompt, return_tensors="pt").input_ids.to(device)
            eval_response = evaluator_model.generate(eval_input_ids, max_new_tokens=10, num_return_sequences=1)
            eval_generated_text = tokenizer_eval.decode(eval_response[0], skip_special_tokens=True)
            is_valid = process_evaluation(eval_generated_text)

            total_moves += 1

            if not is_valid:
                print(f"Jugador {game_instance.current_turn} realizó una jugada inválida en {move}.")
                invalid_moves += 1
                game_over = True
                break

            # Si la jugada es válida, actualiza el tablero
            game_instance.playMove(move, game_instance.current_turn)

            # Revisar si hay un ganador
            if game_instance._checkWin(game_instance.current_turn):
                print(f"¡Jugador {game_instance.current_turn} ganó!")
                if game_instance.current_turn == 'X':
                    wins_p1 += 1
                else:
                    wins_p2 += 1
                game_over = True
            elif game_instance.isFull():
                print("El juego terminó en empate.")
                game_over = True
            else:
                # Cambiar turno
                game_instance.current_turn = opponent

    # Resumen de los resultados
    print(f"\nResultados: \nJugador 1 (X) ganó {wins_p1} veces.")
    print(f"Jugador 2 (O) ganó {wins_p2} veces.")
    print(f"Juegos con jugadas inválidas: {invalid_moves}")
    print(f"Juegos jugados en total: {total_moves}")

# Ejecuta la simulación de 5 juegos
simulate_games(5, model_p1, model_p2, model_eval)


OutOfMemoryError: CUDA out of memory. Tried to allocate 980.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 5.06 MiB is free. Process 25001 has 14.74 GiB memory in use. Of the allocated memory 14.60 GiB is allocated by PyTorch, and 42.88 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [1]:
import random

# Clase TicTacToe para manejar el juego
class TicTacToe:
    def __init__(self):
        self.board = [' ' for _ in range(9)]  # 9 celdas vacías
        self.current_turn = 'X'  # 'X' empieza primero

    def _printBoard(self):
        for i in range(0, 9, 3):
            print(f"{self.board[i]} | {self.board[i+1]} | {self.board[i+2]}")
            if i < 6:
                print("---------")

    def _checkWin(self, mark):
        win_conditions = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # Filas
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # Columnas
            [0, 4, 8], [2, 4, 6]             # Diagonales
        ]
        for condition in win_conditions:
            if all(self.board[i] == mark for i in condition):
                return True
        return False

    def _getEmptyCells(self):
        return [i for i, cell in enumerate(self.board) if cell == ' ']

    def playMove(self, position, mark):
        if self.board[position] == ' ':
            self.board[position] = mark
            return True
        return False

    def isFull(self):
        return ' ' not in self.board





def evaluator_prompt(board, move, player):
    """
    Genera el prompt para el modelo evaluador basado en el estado del tablero y la jugada.
    """
    board_str = "\n".join([" | ".join(board[i:i+3]) for i in range(0, 9, 3)])
    prompt = (
        f"Estado del tablero:\n{board_str}\n\n"
        f"El jugador '{player}' propone jugar en la posición {move}.\n"
        f"Una jugada válida debe ser un número entre 0 y 8 y la celda correspondiente debe estar vacía.\n"
        f"¿Es esta una jugada válida considerando el tablero actual? "
        f"Responde con 'válida' o 'inválida'."
    )
    return prompt

def create_player_prompt(board, current_player, opponent_player):

    # Convertir el tablero en una representación legible
    board_str = '\n'.join([
        f"{board[i]} | {board[i+1]} | {board[i+2]}" if i < 6 else f"{board[i]} | {board[i+1]} | {board[i+2]}"
        for i in range(0, 9, 3)
    ])

    # Generar el prompt para el jugador
    prompt = f"""
    Eres un jugador experto de Tic Tac Toe. El tablero actual es el siguiente:
    {board_str}

    Tu jugada es como el jugador {current_player}. El jugador {opponent_player} ya ha jugado.

    Usa este estado del tablero para tomar una decisión sobre tu siguiente movimiento.
    Tu tarea es escoger una posición vacía entre 0 y 8 para hacer tu jugada.
    """

    return prompt

import torch
from transformers import pipeline
from google.colab import userdata



# # Función para procesar la salida del modelo y extraer la jugada
def process_move(model_output):
    # Aquí puedes ajustar la forma de extraer la jugada según cómo el modelo devuelva los datos
    # Supongamos que el modelo devuelve un texto que contiene un número entre 0 y 8
    move = None
    for word in model_output.split():
        if word.isdigit() and 0 <= int(word) <= 8:
            move = int(word)
            break
    return move

# # Función para procesar la salida del evaluador y determinar si la jugada es válida
def process_evaluation(eval_output):
    # Supongamos que el modelo devuelve "válida" o "inválida"
    if "válida" in eval_output.lower():
        return True
    elif "inválida" in eval_output.lower():
        return False
    return False


from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM


# Inicialización de modelos
hf_token = userdata.get('HF_TOKEN')  # jugador 1
hf_token2 = userdata.get('HF_TOKEN2')  # jugador 2
ehf_token = userdata.get('EHF_TOKEN') # evaluador

model_name_player1 = "bigscience/bloom-560m"
model_name_player2 = "TinyLlama/TinyLlama_v1.1"
model_name_evaluator = "distilgpt2"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Cargar modelos
tokenizer_p1 = AutoTokenizer.from_pretrained(model_name_player1, token=hf_token)
model_p1 = AutoModelForCausalLM.from_pretrained(model_name_player1, token=hf_token).to(device)

if model_p1.config.pad_token_id is None:
    model_p1.config.pad_token_id = model_p1.config.eos_token_id


tokenizer_p2 = AutoTokenizer.from_pretrained(model_name_player2, token=hf_token2)
model_p2 = AutoModelForCausalLM.from_pretrained(model_name_player2, token=hf_token2).to(device)

tokenizer_eval = AutoTokenizer.from_pretrained(model_name_evaluator, token=ehf_token)
model_eval = AutoModelForCausalLM.from_pretrained(model_name_evaluator, token=ehf_token).to(device)


def validate_player_output(output, board):
    move = process_move(output)  # Extraer el número
    if move is None or move not in range(9):
        return False, "La jugada no es un número válido."
    if board[move] != ' ':
        return False, f"La posición {move} ya está ocupada."
    return True, move


def simulate_games(n_games, player1_model, player2_model, evaluator_model):
    wins_p1 = 0
    wins_p2 = 0
    invalid_moves_p1 = 0
    invalid_moves_p2 = 0
    total_moves_p1 = 0
    total_moves_p2 = 0
    valid_moves_p1 = 0
    valid_moves_p2 = 0

    for game in range(n_games):
        game_instance = TicTacToe()
        game_over = False

        print(f"\n--- Juego {game + 1} ---")
        while not game_over:
            # Determinar el modelo y tokenizer del jugador actual
            current_player_model = player2_model if game_instance.current_turn == 'X' else player1_model
            current_tokenizer = tokenizer_p2 if game_instance.current_turn == 'X' else tokenizer_p1
            opponent = 'O' if game_instance.current_turn == 'X' else 'X'

            # Crear el prompt para el jugador actual
            prompt = create_player_prompt(game_instance.board, game_instance.current_turn, opponent)
            input_ids = current_tokenizer(
                prompt,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=512
            ).input_ids.to(device)

            # Generar el movimiento
            move_response = current_player_model.generate(
                input_ids,
                max_new_tokens=10,
                num_return_sequences=1,
                pad_token_id=current_player_model.config.pad_token_id
            )
            generated_text = current_tokenizer.decode(move_response[0], skip_special_tokens=True)

            # Validar la jugada
            is_valid, move = validate_player_output(generated_text, game_instance.board)
            if not is_valid:
                print(f"Jugador {game_instance.current_turn} generó una jugada inválida: {generated_text}")
                game_over = True
                if game_instance.current_turn == 'X':
                    invalid_moves_p1 += 1
                else:
                    invalid_moves_p2 += 1
                continue

            eval_prompt = evaluator_prompt(game_instance.board, move, game_instance.current_turn)
            eval_inputs = tokenizer_eval(
                eval_prompt,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=512
            ).to(device)

            eval_response = model_eval.generate(
                input_ids=eval_inputs.input_ids,
                attention_mask=eval_inputs.attention_mask,
                max_new_tokens=50,
                pad_token_id=tokenizer_eval.pad_token_id
            )
            eval_generated_text = tokenizer_eval.decode(eval_response[0], skip_special_tokens=True)
            is_valid = process_evaluation(eval_generated_text)

            if not is_valid:
                print(f"Jugador {game_instance.current_turn} realizó una jugada inválida en {move}: {eval_generated_text}")
                if game_instance.current_turn == 'X':
                    invalid_moves_p1 += 1
                else:
                    invalid_moves_p2 += 1
                game_over = True
                continue

            # Aplicar la jugada al tablero
            game_instance.playMove(move, game_instance.current_turn)
            if game_instance.current_turn == 'X':
                valid_moves_p1 += 1
                total_moves_p1 += 1
            else:
                valid_moves_p2 += 1
                total_moves_p2 += 1

            # Revisar si hay un ganador
            if game_instance._checkWin(game_instance.current_turn):
                print(f"¡Jugador {game_instance.current_turn} ganó!")
                if game_instance.current_turn == 'X':
                    wins_p1 += 1
                else:
                    wins_p2 += 1
                game_over = True
            elif game_instance.isFull():
                print("El juego terminó en empate.")
                game_over = True
            else:
                # Cambiar turno
                game_instance.current_turn = opponent

    # Resumen de la simulación
    print("\n--- Resumen de la simulación ---")
    print(f"Juegos totales: {n_games}")
    print(f"Ganó jugador 1 (X): {wins_p1}")
    print(f"Ganó jugador 2 (O): {wins_p2}")
    print(f"Juegos con movimientos inválidos: {invalid_moves_p1 + invalid_moves_p2}")
    print(f"Movimientos totales realizados por jugador 1 (X): {total_moves_p1}")
    print(f"Movimientos totales realizados por jugador 2 (O): {total_moves_p2}")
    print(f"Movimientos válidos realizados por jugador 1 (X): {valid_moves_p1}")
    print(f"Movimientos válidos realizados por jugador 2 (O): {valid_moves_p2}")
    print(f"Movimientos inválidos realizados por jugador 1 (X): {invalid_moves_p1}")
    print(f"Movimientos inválidos realizados por jugador 2 (O): {invalid_moves_p2}")



# Asegurar que el token de padding esté definido para cada tokenizador
if tokenizer_p1.pad_token is None:
    tokenizer_p1.pad_token = tokenizer_p1.eos_token

if tokenizer_p2.pad_token is None:
    tokenizer_p2.pad_token = tokenizer_p2.eos_token

if tokenizer_eval.pad_token is None:
    tokenizer_eval.pad_token = tokenizer_eval.eos_token

# Ejecutar la simulación
simulate_games(5, model_p1, model_p2, model_eval)

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

...

Jugador O generó una jugada inválida: 
    Eres un jugador experto de Tic Tac Toe. El tablero actual es el siguiente:
    X |   |  
  |   |  
  |   |  

    Tu jugada es como el jugador O. El jugador X ya ha jugado.

    Usa este estado del tablero para tomar una decisión sobre tu siguiente movimiento.
    Tu tarea es escoger una posición vacía entre 0 y 8 para hacer tu jugada.
    (0 = No hay movimiento, 8 = movimiento

--- Juego 3 ---


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Jugador O generó una jugada inválida: 
    Eres un jugador experto de Tic Tac Toe. El tablero actual es el siguiente:
    X |   |  
  |   |  
  |   |  

    Tu jugada es como el jugador O. El jugador X ya ha jugado.

    Usa este estado del tablero para tomar una decisión sobre tu siguiente movimiento.
    Tu tarea es escoger una posición vacía entre 0 y 8 para hacer tu jugada.
    (0 = No hay movimiento, 8 = movimiento

--- Juego 4 ---


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Jugador O generó una jugada inválida: 
    Eres un jugador experto de Tic Tac Toe. El tablero actual es el siguiente:
    X |   |  
  |   |  
  |   |  

    Tu jugada es como el jugador O. El jugador X ya ha jugado.

    Usa este estado del tablero para tomar una decisión sobre tu siguiente movimiento.
    Tu tarea es escoger una posición vacía entre 0 y 8 para hacer tu jugada.
    (0 = No hay movimiento, 8 = movimiento

--- Juego 5 ---
Jugador O generó una jugada inválida: 
    Eres un jugador experto de Tic Tac Toe. El tablero actual es el siguiente:
    X |   |  
  |   |  
  |   |  

    Tu jugada es como el jugador O. El jugador X ya ha jugado.

    Usa este estado del tablero para tomar una decisión sobre tu siguiente movimiento.
    Tu tarea es escoger una posición vacía entre 0 y 8 para hacer tu jugada.
    (0 = No hay movimiento, 8 = movimiento

--- Resumen de la simulación ---
Juegos totales: 5
Ganó jugador 1 (X): 0
Ganó jugador 2 (O): 0
Juegos con movimientos inválidos: 5
Mov